In [79]:
import time
import re

In [80]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [81]:
wd = "C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver.exe"
addr = "https://news.nate.com/view/20200413n16162?mid=n1008"    # 기사 주소 넣기
addr_nums = re.findall("\d+", addr)
addr_comment = 'http://comm.news.nate.com/Comment/ArticleComment/List?artc_sq='+addr_nums[0]+'n'+addr_nums[1] # 댓글창 주소

In [82]:
driver = webdriver.Chrome(wd)
driver.get(addr_comment)

In [83]:
pages = 2

In [84]:
try:
    for i in range(5):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"body > div.commentBox.reply_hide > div:nth-child(3) > div.paging_wrap > a:nth-child("+str(pages)+")"))).click()
        time.sleep(1)
        print(pages, end=" ")
        pages+=1
except exceptions.ElementNotVisibleException as e: # 페이지 끝
    pass
except Exception as e: # 다른 예외 발생시 확인
    print(e)

2 3 4 5 6 

In [85]:
html = driver.page_source
dom = BeautifulSoup(html, "lxml")

In [86]:
# 댓글이 들어있는 페이지 전체 크롤링
comments_raw = dom.find_all('dd',{'class':'usertxt'})
# 댓글의 text만 뽑는다.
comments = [comment.text for comment in comments_raw]

In [87]:
comments[:10]

['\n하정우 배우로선 명품배우고.\n걷기 같은걸로 스트레스도 건강하게 풀고\n그런다고 생각했었는데.\n다들 힘든것 같다.\n없는 사람도.많이 가진 사람도...\n무엇도 강요될순 없겠지만.\n똑같이 힘들어도 재능과 큰 부가 있다는건\n진짜 신의 엄청난 축복인걸 알고.\n스스로를 바닥까지 끌어 내리시진 말길.\t\t\t ',
 '\n저희도 장난인 줄 알았어요 프로포폴\t\t\t ',
 '\n광고볼때마다 그생각나.포포리\t\t\t ',
 '\n억울해도 개돼지들한테 걸렸으니 앞으로 프로포폴 이미지 계속 가겠네. \n기사 안 읽고 쓴거 같은 댓글들이 배댓 점령한거 보면.. 쟤들은 검찰 경찰에서 아니라고 해도 끝까지 유죄추정을 넘어 유죄판결한다. ㅋㅋㅋ\t\t\t ',
 '\n푹쉬셔~\t\t\t ',
 '\n약중독이 심각한 문젠데\t\t\t ',
 '\n\t\t\t하정우 노무현\t\t\t ',
 '\n\t\t\t프로포폴 처벌 받아야하는거는 맞지만 \n같은 마약사범인 주지훈은 쥰나 빨아제끼면서 하정우는 또 쥰나 까는게 아이러니...\t\t\t ',
 '\n그럴듯하게 컨셉 잘잡고 국민이미지로왓엇는데 \n알맹이까보니 영 아닌거같네\t\t\t ',
 '\nn번방 연예인은 누구게~~???\t\t\t ']

In [88]:
normalized_text = []
for string in comments:
    try:
        tokens = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]+", " ",string)
    except Exception as e:
        print(string)
        break
    normalized_text.append(tokens)
print(normalized_text[:5])

[' 하정우 배우로선 명품배우고 걷기 같은걸로 스트레스도 건강하게 풀고 그런다고 생각했었는데 다들 힘든것 같다 없는 사람도 많이 가진 사람도 무엇도 강요될순 없겠지만 똑같이 힘들어도 재능과 큰 부가 있다는건 진짜 신의 엄청난 축복인걸 알고 스스로를 바닥까지 끌어 내리시진 말길 ', ' 저희도 장난인 줄 알았어요 프로포폴 ', ' 광고볼때마다 그생각나 포포리 ', ' 억울해도 개돼지들한테 걸렸으니 앞으로 프로포폴 이미지 계속 가겠네 기사 안 읽고 쓴거 같은 댓글들이 배댓 점령한거 보면 쟤들은 검찰 경찰에서 아니라고 해도 끝까지 유죄추정을 넘어 유죄판결한다 ㅋㅋㅋ ', ' 푹쉬셔 ']


In [89]:
p_comment_list = []

In [90]:
for comment in comments:
    p_comment_list.append(normalized_text)

In [91]:
file = open('new6.txt','w',encoding = 'utf-8')
for comment in normalized_text:
    file.write(comment + '\n')
file.close()